In [2]:
import pickle
import gzip
import numpy as np, pandas as pd

In [7]:
def get_filenames():
    files = !ls
    files = [f for f in files if 'atis' in f]
    return files
get_filenames()

['atis.fold0.pkl.gz',
 'atis.fold1.pkl.gz',
 'atis.fold2.pkl.gz',
 'atis.fold3.pkl.gz',
 'atis.fold4.pkl.gz']

In [15]:
iterables = [['Word1', 'Word2'], ['word_length', 'is_uppercase?', 'prev_token_is_city']]
index = pd.MultiIndex.from_product(iterables, names=['first', 'second'])
df = pd.DataFrame(np.random.randn(3, 6), index=['A', 'B', 'C'], columns=index)
df.head()

first        Word1                                        Word2                \
second word_length is_uppercase? prev_token_is_city word_length is_uppercase?   
A        -0.163653     -0.930804          -0.694448   -0.975145     -0.962460   
B         0.448409     -1.244464          -0.280922    1.292447      1.984015   
C        -0.315218      0.549252          -0.945447   -0.715877      0.665729   

first                      
second prev_token_is_city  
A               -0.294057  
B               -1.624580  
C                0.727129

In [6]:
def read_file(filename):
    f = gzip.open(filename, 'rb')
    try:
        train_set, valid_set, test_set, dicts = pickle.load(f, encoding='latin1')
    except:
        train_set, valid_set, test_set, dicts = pickle.load(f)
        
    return train_set, valid_set, test_set, dicts

train_set, _, _, dicts = read_file(get_filenames()[0])
print(len(train_set[0]))
print(train_set[0][0])
print(train_set[1][0])
print(train_set[2][0])

3983
[554 194 268  64  62  16   8 234 481  20  40  58 234 415 205]
[  0   0   0  18   0   1  52   0   0  76   0   0   0  18 109]
[126 126 126  48 126  36  35 126 126  33 126 126 126  78 123]


In [5]:
def get_sample_data(how_many, filename, max_length=None):
    words_train_set, _, _, dicts = read_file(filename)
    
    words2idx = dicts['words2idx']
    labels2idx = dicts['labels2idx']
    tables2idx = dicts['tables2idx']
    
    idx2words = {words2idx[k]: k for k in words2idx}
    idx2labels = {labels2idx[k]: k for k in labels2idx}
    idx2tables = {tables2idx[k]: k for k in tables2idx}
    
    sample_data = []
    for i in range(how_many*5):
        if max_length is not None:
            if len(train_set[0][i]) > max_length:
                continue
        sample_data.append(
            (
                " ".join([idx2words[idx] for idx in train_set[0][i]]),
                " ".join([idx2labels[idx] for idx in train_set[2][i]])
            )
        )
        
    return sample_data[:how_many]

get_sample_data(50, get_filenames()[0], 10)

[('what is the abbreviation for canadian airlines international',
  'O O O O O B-airline_name I-airline_name I-airline_name'),
 ('show me the us air flights from atlanta to boston',
  'O O O B-airline_name I-airline_name O O B-fromloc.city_name O B-toloc.city_name'),
 ('show me the cheapest round trips from dallas to baltimore',
  'O O O B-cost_relative B-round_trip I-round_trip O B-fromloc.city_name O B-toloc.city_name'),
 ('explain fare code qx', 'O O O B-fare_basis_code'),
 ('what does the meal code s stand for', 'O O O B-meal O B-meal_code O O'),
 ('what are all flights to denver from philadelphia on sunday',
  'O O O O O B-toloc.city_name O B-fromloc.city_name O B-depart_date.day_name'),
 ('what flights are available monday from san francisco to pittsburgh',
  'O O O O B-depart_date.day_name O B-fromloc.city_name I-fromloc.city_name O B-toloc.city_name'),
 ('what airlines have business class', 'O O O B-class_type I-class_type'),
 ('flights from atlanta to washington dc',
  'O O B-